# Scala notebook

In [ ]:
import $ivy.`org.apache.spark::spark-sql:3.1.2`
import $ivy.`sh.almond::ammonite-spark:0.1.3`
import $ivy.`io.delta:delta-core_2.12:1.0.0`
import $ivy.`io.delta:delta-sharing-spark_2.12:0.1.0`
import $ivy.`com.amazonaws:aws-java-sdk-bundle:1.11.563`
import $ivy.`org.apache.hadoop:hadoop-aws:3.2.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

In [ ]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .config("spark.hadoop.fs.s3a.access.key", sys.env.get("AWS_ACCESS_KEY_ID").getOrElse("test"))
    .config("spark.hadoop.fs.s3a.secret.key", sys.env.get("AWS_SECRET_ACCESS_KEY").getOrElse("test"))
    .config("spark.hadoop.fs.s3a.endpoint", s"""${sys.env.get("AWS_SERVER").getOrElse("amazon.com")}:${sys.env.get("AWS_PORT").getOrElse("8000")}""")
    .getOrCreate()
}

In [ ]:
val citiesDF = spark.read.options(Map("header"->"true", "inferSchema"->"true")).csv("s3://demodata/rawdata/world/cities/sample.csv")
citiesDF.show(truncate=false)
citiesDF.printSchema()


In [ ]:
citiesDF.write.format("delta").save("s3://demodata/silver/world/cities")
